In [37]:
from SPARQLWrapper import SPARQLWrapper, JSON
from pandas import json_normalize

endpoint = SPARQLWrapper("http://localhost:8890/sparql")
endpoint.setReturnFormat(JSON)
endpoint.setTimeout(1200)
endpoint.method = 'POST'

def run_query(query, endpoint, as_dataframe=False, do_print=False):
    PREFIX= """

    """
    
    endpoint.setQuery(PREFIX+query)
    results = endpoint.query().convert()['results']
    if len(results['bindings']) <= 0:
        print("Empty resultset")
        
    if not as_dataframe:
        if do_print:
            for binding in results['bindings']:    
                print("; ".join([var+": "+ binding[var]['value']  for var in binding.keys()  ]))
        return results['bindings']
    
    else:
        # pdata = pd.DataFrame.from_dict(results['bindings'], orient="index")
        pdata = json_normalize(results['bindings'])
        if do_print:
            display(pdata)
        return pdata

In [38]:
query = 'SELECT ?s ?p ?o FROM <http://www.yelpkg.com/business> WHERE{?s ?p ?o } LIMIT 10'
run_query(query=query, endpoint=endpoint, as_dataframe=False, do_print=True)

URLError: <urlopen error [Errno 61] Connection refused>